<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/Database_08_IndexesTransactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Database Performance: Index, Query Plans, and Transactsion Management

In this chapter, we'll be exploring the inner workings of **SQLite** using a dataset from the fictional **Gotham National Bank**. Our database, *gotham.db*, contains two tables:

1.  **Customers**: Stores customer information like name, address, and contact details.
2.  **Accounts**: Holds account-related data such as account type, balance, and creation date, with a foreign key linking to the Customers table.

Throughout this chapter, we'll use these tables to understand how **indexes** and **transactions** can optimize our SQLite database performance. We'll cover the fundamentals of indexes, demonstrate their impact on query speed and database size, and discuss best practices for implementing them effectively.

But before we dive in, let's take a closer look at our dataset. The Customers table includes an intriguing cast of characters, from the enigmatic Bruce Wayne to the clever Selina Kyle. Each customer has a unique *CustomerID*, serving as the **primary key**.

The Accounts table, on the other hand, logs the financial details of each customer's bank accounts. It uses *AccountID* as its primary key and *CustomerID* as a **foreign key** to establish a link between the two tables. This allows us to associate each account with its respective owner.

In [1]:
!wget https://github.com/brendanpshea/database_sql/raw/main/data/gotham.db -q -nc
%load_ext sql
%sql sqlite:///gotham.db


## Database Schema for Gotham City Bank
Here is the database schema.

In [2]:
%%sql
-- get table schema using sqlite master
SELECT sql FROM sqlite_master WHERE type='table';

 * sqlite:///gotham.db
Done.


sql
"CREATE TABLE Customers ( CustomerID INTEGER PRIMARY KEY AUTOINCREMENT, FirstName VARCHAR(50) NOT NULL, LastName VARCHAR(50) NOT NULL, MiddleInitial CHAR(1), Address VARCHAR(255), City VARCHAR(50), State VARCHAR(50), ZipCode VARCHAR(10), PhoneNumber VARCHAR(15), Email VARCHAR(100))"
"CREATE TABLE sqlite_sequence(name,seq)"
"CREATE TABLE Accounts ( AccountID INTEGER PRIMARY KEY AUTOINCREMENT, CustomerID INT, AccountType VARCHAR(20), Balance DECIMAL(15, 2) DEFAULT 0.00, CreatedDate DATE, FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID))"


## A Closer Look at Customers and Accounts
Now, let's take a quick look at the head of our table.

In [3]:
%%sql
SELECT * FROM Customers LIMIT 5;

 * sqlite:///gotham.db
Done.


CustomerID,FirstName,LastName,MiddleInitial,Address,City,State,ZipCode,PhoneNumber,Email
1,Bruce,Wayne,None,1007 Mountain Drive,Gotham,NY,10001,123-456-7890,bruce.wayne@gothambank.com
2,Selina,Kyle,None,123 Cat St,Blüdhaven,NY,20002,234-567-8901,selina.kyle@gothambank.com
3,James,Gordon,None,789 Police Plaza,Gotham,NY,10003,345-678-9012,james.gordon@gothambank.com
4,Harvey,Dent,None,456 Lawyer Ave,Metropolis,NY,30004,456-789-0123,harvey.dent@gothambank.com
5,Pamela,Isley,None,369 Botanic St,Gotham,NY,10005,567-890-1234,pamela.isley@gothambank.com


In [4]:
%%sql
SELECT * FROM Accounts LIMIT 5;

 * sqlite:///gotham.db
Done.


AccountID,CustomerID,AccountType,Balance,CreatedDate
1,1,Checking,15000,2021-01-10
2,1,Savings,75000,2021-02-11
3,2,Checking,32000,2022-03-05
4,3,Checking,86000,2023-04-15
5,4,Checking,22000,2024-05-20


## What are Indexes and How Do I Create Them?

If you've ever used a book index to quickly locate a specific topic, you already understand the basic concept of a database **index**. In the context of databases, an index is a separate data structure that allows the database engine to find and retrieve data more efficiently without having to scan the entire table.

When you create a table, SQLite (like all versions of SQL) automatically creates a unique index on the **primary key** column(s). However, you can also create additional indexes on other columns to speed up frequently used queries.

To create an index in SQL, you use the **CREATE INDEX** statement with the following syntax:

```sql
CREATE [UNIQUE] INDEX index_name
ON table_name (column1 [, column2, ...]);
```

Here's a breakdown of the components:

-   `UNIQUE`: Optional keyword that creates a unique index, ensuring no duplicate values in the indexed column(s).
-   `index_name`: The name you want to give your index.
-   `table_name`: The name of the table on which you're creating the index.
-   `column1, column2, ...`: The column(s) to be indexed. You can index multiple columns by separating them with commas.

For example, let's say we want to create an index on the *LastName* column of our *Customers* table:!

In [5]:
%%sql
DROP INDEX IF EXISTS idx_customers_lastname;
CREATE INDEX idx_customers_lastname
ON Customers (LastName);

 * sqlite:///gotham.db
Done.
Done.


[]

This statement creates an index named idx_customer_lastname on the LastName column, allowing for faster searches based on customer last names. This will allow us to search customers by their last name more quickly.

SQLite supports several types of indexes, including unique indexes, which ensure that the indexed column(s) contain no duplicate values, and partial indexes, which only index a subset of rows based on a specified condition.

Creating appropriate indexes is crucial for optimizing database performance, but how do they actually impact query speed? In the next section, we'll explore the power of indexes in action using the EXPLAIN QUERY PLAN command and some practical examples from our Gotham City dataset.

## How Indexes Impact Speed

To understand the impact of indexes on query speed, we'll use the **EXPLAIN QUERY PLAN** command. This command provides insight into how SQLite executes a query, including which indexes (if any) are being used.

Let's start with an example query on our *Accounts* table that gets customers with higher than average balances

In [6]:
%%sql
SELECT * FROM Accounts
WHERE Balance > (SELECT AVG(Balance) FROM Accounts)
ORDER BY Balance DESC
LIMIT 5;

 * sqlite:///gotham.db
Done.


AccountID,CustomerID,AccountType,Balance,CreatedDate
8136,7952,Savings,92802.15340000001,2003-01-18
3401,4699,Savings,91534.3495,2015-03-25
304,277,Savings,91283.00280000002,2021-05-08
10,8,Checking,91000,2024-10-10
5953,5751,Savings,90689.35260000001,2013-12-29


Now, let's see what the "query plan" is for this:

In [7]:
%%sql
DROP INDEX IF EXISTS idx_accounts_balance;

EXPLAIN QUERY PLAN
SELECT * FROM Accounts
WHERE Balance > (SELECT AVG(Balance) FROM Accounts)
ORDER BY Balance DESC;

 * sqlite:///gotham.db
Done.
Done.


id,parent,notused,detail
3,0,0,SCAN Accounts
8,0,0,SCALAR SUBQUERY 1
13,8,0,SCAN Accounts
31,0,0,USE TEMP B-TREE FOR ORDER BY


If you look at the query plan here, you'll notice a few key terms that pop out:

-   **Full Table Scan**: A full table scan occurs when SQLite reads every row in a table to find the data that matches the query conditions. In this case, `SCAN Accounts` indicates that SQLite is performing a full table scan on the *Accounts* table. Full table scans can be inefficient, especially for large tables, as they require reading and checking every single row. This will change when we create an index.
-   **Scalar Subquery**: A scalar subquery is a subquery that returns a single value. In this example, `SCALAR SUBQUERY 1` refers to the subquery `(SELECT AVG(Balance) FROM Accounts)`, which calculates the average balance of all accounts. The result of this subquery is used in the main query's WHERE clause.
-   **Temporary B-Tree**: In this case, `USE TEMP B-TREE FOR ORDER BY` indicates that SQLite is using a temporary B-tree data structure to sort the result set in descending order by the *Balance* column. We'll dive deeper into B-trees in a future section.

### Impact of Indexes

If an index were created on the `Balance` column, the query execution plan would likely be different. For example, with an index on `Balance`, the execution plan might show an **index search** instead of a full table scan, resulting in faster query execution times.

To create an index on the `Balance` column, you could use the following command:

In [8]:
%%sql
CREATE INDEX idx_accounts_balance
ON Accounts (Balance);

 * sqlite:///gotham.db
Done.


[]

Now, let's see what happens to our query plan:

In [9]:
%%sql
EXPLAIN QUERY PLAN
SELECT * FROM Accounts
WHERE Balance > (SELECT AVG(Balance) FROM Accounts)
ORDER BY Balance DESC;

 * sqlite:///gotham.db
Done.


id,parent,notused,detail
4,0,0,SEARCH Accounts USING INDEX idx_accounts_balance (Balance>?)
8,0,0,SCALAR SUBQUERY 1
13,8,0,SCAN Accounts USING COVERING INDEX idx_accounts_balance


SQLite now uses the idx_account_balance index to search for matching rows, resulting in a more efficient query execution. Instead of scanning the entire table, it can quickly locate the relevant rows using the index.

Some new terms you'll notice:

1.  **Index Scan**: An index scan is a type of table access that uses an index to locate the required data. In this example, `SEARCH Accounts USING INDEX idx_accounts_balance (Balance>?)` means that SQLite is using the *idx_accounts_balance* index to find rows where the *Balance* is greater than the result of the scalar subquery.
2.  **Covering Index**: A covering index is an index that contains all the columns required to satisfy a query, eliminating the need for additional table lookups. In the updated plan, `SCAN Accounts USING COVERING INDEX idx_accounts_balance` indicates that the *idx_accounts_balance* index is a covering index for the subquery. This means that all the data needed to calculate the average balance is available within the index itself, making the query more efficient.


## Timing Our Queries
To further demonstrate the performance impact, let's measure the execution time of our query with and without the index. We'll be using Colab's builtin **timeit** function, which will run this query many times, and give us the average runninng time. This helps miminize the role of chance in evaluating query performance based on a single case.

In [10]:
%%sql
-- to start, drop the index
DROP INDEX idx_accounts_balance;

 * sqlite:///gotham.db
Done.


[]

In [11]:
%%timeit -n 1
%%sql
-- This will run this query many times
SELECT * FROM Accounts
WHERE Balance > (SELECT AVG(Balance) FROM Accounts)
ORDER BY Balance DESC;

 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
66 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%sql
-- Recreate our index
CREATE INDEX idx_accounts_balance
ON Accounts (Balance);

 * sqlite:///gotham.db
Done.


[]

In [13]:
%%timeit -n 1
%%sql
-- Now run the query with the index
SELECT * FROM Accounts
WHERE Balance > 25000 AND  Balance < 50000;

 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
 * sqlite:///gotham.db
Done.
47.7 ms ± 9.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


While the specific results you get will vary each time you run the test, you should find that (on average) the index scan runs a bit faster than the full table scan.

## How Indexes Impact Storage
Now that we've seen how indexes can significantly improve query performance, let's explore how they impact database storage. To understand this, we first need to discuss how SQLite (and many other databases) store data using a data structure called a B-tree.

### B-trees: The Backbone of Database Storage

A B-tree is a self-balancing tree data structure that maintains sorted data and allows for efficient insertion, deletion, and search operations. B-trees are particularly well-suited for databases because they optimize disk I/O operations, which is crucial when dealing with large amounts of data that cannot fit entirely in memory.

In a B-tree, each node contains multiple keys and pointers to child nodes. The keys within a node are kept in sorted order, and each key is associated with two pointers:

1.  A pointer to the child node containing keys less than the current key.
2.  A pointer to the child node containing keys greater than or equal to the current key.

This structure allows for quick traversal and search operations, as the database can navigate the tree by comparing the desired key with the keys in each node and following the appropriate pointers.

### How SQLite Uses B-trees

SQLite uses B-trees to store both tables and indexes. When you create a table, SQLite creates a B-tree where each node contains one or more rows of data. The rows within a node are sorted by the primary key, enabling efficient lookups and range queries.

For example, let's consider a simplified representation of the *Accounts* table:

```
AccountID | CustomerID | Balance
1         | 1          | 10000
2         | 1          | 5000
3         | 2          | 7500
4         | 3          | 12000`
```

In the table's B-tree, the rows would be stored in nodes, sorted by the primary key (AccountID):

```
Node 1: (1, 1, 10000) | (2, 1, 5000)
Node 2: (3, 2, 7500)  | (4, 3, 12000)
```

When you create an index on a column, SQLite generates a separate B-tree specifically for that index. The index B-tree stores the indexed column's values along with pointers to the corresponding rows in the table's B-tree.

Let's say we create an index on the *Balance* column. Then the *idx_accounts_balance* index B-tree would store the *Balance* values in sorted order, along with pointers to the corresponding rows in the *Accounts* table's B-tree:

```sql
Node 1: (5000, pointer to row 2) | (7500, pointer to row 3)
Node 2: (10000, pointer to row 1) | (12000, pointer to row 4)
```

When you query the table using the indexed column, SQLite can quickly traverse the index B-tree to find the matching values and then follow the pointers to retrieve the full row data from the table's B-tree.

By using B-trees for both tables and indexes, SQLite ensures efficient data storage and retrieval, even for large datasets. However, it's important to keep in mind that creating indexes also requires additional storage space, as each index maintains its own B-tree structure.


### Graphic: B-Trees
Here's what our B-tree for our original table might look like:


In [38]:
import base64
from IPython.display import Image, display, HTML

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))


mm("""
graph TB
  subgraph Accounts Table B-tree - sorting by id
    N1((Node 1)) --- |pointer to next/previous node| N2((Node 2))
    N1 --- |contains| A1[AccountID - key: 1<br>CustomerID: 1<br>Balance: 10000]
    N1 --- |contains| A2[AccountID - key: 2<br>CustomerID: 1<br>Balance: 5000]
    N2 --- |contains| A3[AccountID - key: 3<br>CustomerID: 2<br>Balance: 7500]
    N2 --- |contains| A4[AccountID - key: 4<br>CustomerID: 3<br>Balance: 12000]
  end
""")

And here's what the B-tree for the index might look like:

In [39]:
mm("""
graph TB
  subgraph Balance Index B-tree - sorting by balance
    IN1((Node 1)) --- | points to next/previous node | IN2((Node 2))
    IN1 --- IA2[Balance - key: 5000<br>Pointer to Account 2]
    IN1 --- IA3[Balance - key: 7500<br>Pointer to Account 3]
    IN2 --- IA1[Balance - key: 10000<br>Pointer to Account 1]
    IN2 --- IA4[Balance - key: 12000<br>Pointer to Account 4]
  end
"""
)